<a href="https://colab.research.google.com/github/xaldoxxx/curiosidades/blob/main/peewee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ORM peewee tuto oficial



http://docs.peewee-orm.com/en/latest/




https://www.paradigmadigital.com/dev/como-hacer-bases-datos-con-peewee/


https://gugadev.wordpress.com/about/




In [ ]:
!pip install peewee

     |████████████████████████████████| 850 kB 5.1 MB/s 
  Created wheel for peewee: filename=peewee-3.14.8-cp37-cp37m-linux_x86_64.whl size=614549 sha256=7d71726934205ccc188fca941fbca84f67975b2b7ddd3c926523f86ce72db2b4
  Stored in directory: /root/.cache/pip/wheels/eb/72/f3/8b7b2ae199059c7d6f7686d342c7afdc7b703280a79848defd
Successfully built peewee


In [ ]:
# crea base de datos
from peewee import *
import datetime

try:
    db = SqliteDatabase('nivel_avanzado.db')
    class BaseModel(Model):
        class Meta:
            database = db
    class Noticia(BaseModel):
        titulo = CharField(unique = True)
#        descripcio = TextField()
        fecha = DateTimeField()
        estado_de_publicacion = BooleanField(default=True)
    db.connect()
    db.create_tables([Noticia])
except:
    print("mmmm")

In [ ]:
import peewee

In [ ]:
peewee.DateTimeField

peewee.DateTimeField

In [ ]:
from peewee import SqliteDatabase, AutoField, CharField, DateField, ForeignKeyField, Model

db = SqliteDatabase('academia.db')

class Profesores(Model):
   maestro_id = AutoField()
   nombre = CharField()
   apellido = CharField()
   telefono = CharField()
   email = CharField(unique=True)

   class Meta:
       database = db

class Clases(Model):
   clase_id = AutoField()
   cod_curso = CharField()
   fecha_inicio_curso = DateField()
   fecha_fin_curso = DateField()
   horario = CharField()
   maestro_id = ForeignKeyField(Profesores)

   class Meta:
       database = db

db.connect()
db.create_tables([Profesores, Clases])

In [ ]:
# select
for profesor in Profesores.select():
     print('Nombre: {} - Apellido: {} - Teléfono: {} - Email: {}'
     .format(profesor.nombre, profesor.apellido, profesor.telefono, profesor.email))

In [ ]:
# consulta
query = (Profesores
        .select(Profesores, Clases)
        .join(Clases)
        .group_by(Clases.cod_curso)
        .where(Profesores.maestro_id == Clases.maestro_id))

for curso in query:
   print('El curso {} esta comienza el {} y termina el {}, y va ser impartido por {} {}'
   .format(curso.clases.cod_curso, curso.clases.fecha_inicio_curso, curso.clases.fecha_fin_curso, curso.nombre, curso.apellido))

In [ ]:
# Acá hacemos un JOIN de la tabla de “Profesores” con la tabla “Clases” y seleccionamos todos los campos de ambas tablas, utilizando un WHERE donde el maestro_id es igual en cada tabla, su sentencia SQL sería la siguiente:
SELECT "t1"."maestro_id", "t1"."nombre", "t1"."apellido", "t1"."telefono", "t1"."email", "t2"."clase_id", "t2"."cod_curso", "t2"."fecha_inicio_curso", "t2"."fecha_fin_curso", "t2"."horario", "t2"."maestro_id" FROM "profesores" AS
 "t1" INNER JOIN "clases" AS "t2" ON ("t2"."maestro_id" = "t1"."maestro_id") WHERE ("t1"."maestro_id" = "t2"."maestro_id") GROUP BY "t2"."cod_curso"

In [ ]:
# actualiza
chamo_update = Profesores.update(nombre='Alvaro').where(Profesores.maestro_id == 1).execute()

In [ ]:
# borra
go_delete = Clases.delete().where(Clases.cod_curso == 'go_101').execute()


In [ ]:
from peewee import *

db = SqliteDatabase('people.db')

class Person(Model):
    name = CharField()
    birthday = DateField()

    class Meta:
        database = db # This model uses the "people.db" database.

In [ ]:
class Pet(Model):
    owner = ForeignKeyField(Person, backref='pets')
    name = CharField()
    animal_type = CharField()

    class Meta:
        database = db # this model uses the "people.db" database

In [ ]:
db.connect() # conectamos con la base de datos

True

In [ ]:
db.create_tables([Person, Pet]) # creamos tablas

In [ ]:
from datetime import date

In [ ]:
# cargamos bd
uncle_bob = Person(name='Bob', birthday=date(1960, 1, 15))
uncle_bob.save() # bob is now stored in the database

1

In [ ]:
# otra forma
grandma = Person.create(name='Grandma', birthday=date(1935, 3, 1))
herb = Person.create(name='Herb', birthday=date(1950, 5, 5))

In [ ]:
# actualiza una fila
grandma.name = 'Grandma L.'
grandma.save()  # Update grandma's name in the database.
# Returns: 1

1

In [ ]:
# ya almacenamos tres personas ahora le agregamos mascotas
bob_kitty = Pet.create(owner=uncle_bob, name='Kitty', animal_type='cat')
herb_fido = Pet.create(owner=herb, name='Fido', animal_type='dog')
herb_mittens = Pet.create(owner=herb, name='Mittens', animal_type='cat')
herb_mittens_jr = Pet.create(owner=herb, name='Mittens Jr', animal_type='cat')

In [ ]:
# eliminamos una mascota
herb_mittens.delete_instance() # he had a great life
# Returns: 1

1

In [ ]:
# le agregamos otra mascota a bob
herb_fido.owner = uncle_bob
herb_fido.save()

1

In [ ]:
grandma = Person.select().where(Person.name == 'Grandma L.').get() # obtener datos unicos

In [ ]:
grandma = Person.get(Person.name == 'Grandma L.') # otra forma

In [ ]:
# listar registros
for person in Person.select():
    print(person.name)

Bob
Grandma L.
Herb


In [ ]:
# listar gatos y nombres de los dueños
query = Pet.select().where(Pet.animal_type == 'cat')
for pet in query:
    print(pet.name, pet.owner.name)

Kitty Bob
Mittens Jr Herb


In [ ]:
# evitar las consultas adicionales seleccionando tanto Mascota como Persona , y agregando una combinación
query = (Pet
         .select(Pet, Person)
         .join(Person)
         .where(Pet.animal_type == 'cat'))

for pet in query:
    print(pet.name, pet.owner.name)

Kitty Bob
Mittens Jr Herb


In [ ]:
# todas las mascotas de bob
for pet in Pet.select().join(Person).where(Person.name == 'Bob'):
    print(pet.name)

Kitty
Fido


In [ ]:
# ordenado alfabeticamente
for pet in Pet.select().where(Pet.owner == uncle_bob).order_by(Pet.name):
    print(pet.name)

Fido
Kitty


In [ ]:
# listar personas de menor a mayor
for person in Person.select().order_by(Person.birthday.desc()):
    print(person.name, person.birthday)

Bob 1960-01-15
Herb 1950-05-05
Grandma L. 1935-03-01


In [ ]:
from playhouse.sqlite_ext import SqliteExtDatabase

db = SqliteExtDatabase('my_app.db', pragmas=(
    ('cache_size', -1024 * 64),  # 64MB page-cache.
    ('journal_mode', 'wal'),  # Use WAL-mode (you should always use this!).
    ('foreign_keys', 1)))  # Enforce foreign-key constraints.

In [ ]:
db = CSqliteExtDatabase(':memory:')

@db.on_commit
def on_commit():
    logger.info('COMMITing changes')

In [ ]:
db.close.
